In [1]:
%%init_spark
launcher.jars = ["file:///home/sammy/iceberg-spark-runtime-3.2_2.12-0.13.2.jar"]
launcher.master = "local[4]"
launcher.conf.spark.executor.cores = 8


In [14]:
import scala.collection.JavaConverters._


import scala.collection.JavaConverters._


In [2]:
import org.apache.hadoop.conf.Configuration;
import org.apache.iceberg.hadoop.HadoopCatalog;

val conf = new Configuration();
val warehousePath = "file:///tmp/hdfs/test5";
val catalog = new HadoopCatalog(conf, warehousePath);


Intitializing Scala interpreter ...

Spark Web UI available at http://lambo.localdomain:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[4], app id = local-1655502643152)
SparkSession available as 'spark'


import org.apache.hadoop.conf.Configuration
import org.apache.iceberg.hadoop.HadoopCatalog
conf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-rbf-default.xml, hdfs-site.xml, hdfs-rbf-site.xml
warehousePath: String = file:///tmp/hdfs/test5
catalog: org.apache.iceberg.hadoop.HadoopCatalog = HadoopCatalog{name=hadoop, location=file:///tmp/hdfs/test5}


In [6]:
import org.apache.iceberg.Schema;
import org.apache.iceberg.types.Types;

val schema = new Schema(
      Types.NestedField.required(1, "id", Types.LongType.get()),
      Types.NestedField.required(2, "event_time", Types.TimestampType.withZone()),
    );

import org.apache.iceberg.Schema
import org.apache.iceberg.types.Types
schema: org.apache.iceberg.Schema =
table {
  1: id: required long
  2: event_time: required timestamptz
}


In [26]:
import org.apache.iceberg.catalog.TableIdentifier

import org.apache.iceberg.PartitionSpec

val partSpecBuilder = PartitionSpec.builderFor(schema).hour("event_time")

val partSpec = partSpecBuilder.build()


val table = catalog.buildTable(TableIdentifier.of("hi8"), schema).withPartitionSpec(partSpec)

import org.apache.iceberg.catalog.TableIdentifier
import org.apache.iceberg.PartitionSpec
partSpecBuilder: org.apache.iceberg.PartitionSpec.Builder = org.apache.iceberg.PartitionSpec$Builder@41cd7733
partSpec: org.apache.iceberg.PartitionSpec =
[
  1000: event_time_hour: hour(2)
]
table: org.apache.iceberg.catalog.Catalog.TableBuilder = org.apache.iceberg.hadoop.HadoopCatalog$HadoopCatalogTableBuilder@70731531


In [27]:
val created_table = table.create()

created_table: org.apache.iceberg.Table = hadoop.hi8


import org.apache.iceberg.PartitionSpec
partSpecBuilder: org.apache.iceberg.PartitionSpec.Builder = org.apache.iceberg.PartitionSpec$Builder@7231ade0
partSpec: org.apache.iceberg.PartitionSpec =
[
  1000: event_time_hour: hour(2)
]


In [28]:
import org.apache.iceberg.Metrics
import org.apache.iceberg.types.Conversions
import org.apache.iceberg.DataFiles


for (i <- 0 to 4) {
    val columnSizes = Map[Int, Long](1 -> 100*8, 2 -> 100*8).asJava.asInstanceOf[java.util.Map[java.lang.Integer, java.lang.Long]]
    val valueCounts = Map[Int, Long](1 -> 100, 2 -> 100).asJava.asInstanceOf[java.util.Map[java.lang.Integer, java.lang.Long]]
    val nullValueCounts = Map[Int, Long](1 -> 0, 2 -> 0).asJava.asInstanceOf[java.util.Map[java.lang.Integer, java.lang.Long]]
    val nanValueCounts = Map[Int, Long](1 -> 0, 2 -> 0).asJava.asInstanceOf[java.util.Map[java.lang.Integer, java.lang.Long]]

    val lower_bound = Map[java.lang.Integer, java.nio.ByteBuffer](Int.box(1) -> Conversions.toByteBuffer(Types.LongType.get(), i * 50L)).asJava
    val upper_bound = Map[java.lang.Integer, java.nio.ByteBuffer](Int.box(1) -> Conversions.toByteBuffer(Types.LongType.get(), (i + 1) * 50L)).asJava

    val metrics = new Metrics(
        50,
        columnSizes,
        valueCounts,
        nullValueCounts,
        nanValueCounts,
        lower_bound,
        upper_bound
    )
    val dataFileBuilder = DataFiles.builder(created_table.spec())
    .withPath("file:///tmp/test" + i)
    .withFormat("parquet")
    .withFileSizeInBytes(1000000)
    .withRecordCount(50)
    .withMetrics(metrics)



    val dataFile = dataFileBuilder.build()
    created_table.newAppend().appendFile(dataFile).commit()
    
}

import org.apache.iceberg.Metrics
import org.apache.iceberg.types.Conversions
import org.apache.iceberg.DataFiles


In [29]:
import org.apache.iceberg.expressions.Expressions
val scan = created_table.newScan().filter(Expressions.lessThan("id", 100)).asOfTime(1655528018000L)

for (file <- scan.planTasks().asScala) {
  println(file)
}

BaseCombinedScanTask{tasks=org.apache.iceberg.BaseFileScanTask$SplitScanTask@2c9d49b7, org.apache.iceberg.BaseFileScanTask$SplitScanTask@21c3a8a3}


import org.apache.iceberg.expressions.Expressions
scan: org.apache.iceberg.TableScan = DataTableScan{table=hadoop.hi8, projection=struct<1: id: required long, 2: event_time: required timestamptz>, filter=ref(name="id") < 100, ignoreResiduals=false, caseSensitive=true}


In [30]:

for (file <- scan.planFiles().asScala) {
  println(file)
}

BaseFileScanTask{file=file:///tmp/test1, partition_data=PartitionData{event_time_hour=null}, residual=ref(name="id") < 100}
BaseFileScanTask{file=file:///tmp/test0, partition_data=PartitionData{event_time_hour=null}, residual=ref(name="id") < 100}
